# **Finding Optimal Time Window Size 1, 7, 30, 180 days**

In [5]:
%run bi_feature_eng_utilities.ipynb 
!pip install pyarrow
!pip install dask
#Clean up previous tables
!aws s3 rm --recursive s3://datascience-workspace-dev/buyer-intent/user_interactions/tables

  Using cached https://s3-us-west-2.amazonaws.com/move-dl-common-binary-distrubution/python/move_dl_common_api-3.2.131-release.tar.gz
  Using cached https://files.pythonhosted.org/packages/6b/fd/18c862d197105b6fbe2d27d62ae541bd41e883f4e2b5ab6aab8c9d1a2396/boto3-1.9.202-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/16/8a/1fc3dba0c4923c2a76e1ff0d52b305c44606da63f718d14d3231e21c51b0/s3transfer-0.2.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/70/f2/9e591425a2543a270c7e30551472f425fd31ddc8b117f6cfa6f85c98a068/botocore-1.12.202-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/83/94/7179c3832a6d45b266ddb2aac329e101367fbdb11f425f13771d27f225bb/jmespath-0.9.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/36/fa/08e9e6e0e3cbd1d362c3bbee8d01d0aedb2155c4ac112b19ef3cae8eed8d/docutils-0.14-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/41/17/c62faccbfbd163c7

In [6]:
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pickle
from scipy import stats
from sklearn.cluster import KMeans
from datetime import datetime
import pyarrow.parquet as pq
import dask.dataframe as dd
import s3fs
import multiprocessing

# Random Sample input data if too large this number should be presented as integer ]0,100]
sample_rate_percentage = 100

s3 = s3fs.S3FileSystem()

In [7]:
user_interactions_db_handler= FeatureEngineeringUtilities(db_name= 'buyer_intent_user_interactions', 
                                                          db_path_on_s3 = 's3://datascience-workspace-dev/buyer-intent/user_interactions', 
                                                          logger_name = 'buyer_intent')

2019-08-07 17:53:42.570 INFO athena_util.py.111 - Read config from default schema
2019-08-07 17:53:42.571 INFO athena_util.py.114 - Schema fetched from default config file:


In [8]:
user_interactions_db_handler.drop_table('biui_sampled_{}_consumer_analytical_profile_summary_t180'.format(
                                                                    str(int((100/sample_rate_percentage)*100))))
user_interactions_db_handler.drop_table('biui_sampled_{}_consumer_analytical_profile_summary_t030'.format(
                                                                    str(int((100/sample_rate_percentage)*100))))
user_interactions_db_handler.drop_table('biui_sampled_{}_consumer_analytical_profile_summary_t007'.format(
                                                                    str(int((100/sample_rate_percentage)*100))))
user_interactions_db_handler.drop_table('biui_sampled_{}_consumer_analytical_profile_summary_t001'.format(
                                                                    str(int((100/sample_rate_percentage)*100))))

user_interactions_db_handler.random_sample_based_on_string_attr(from_table_name = 'biui_consumer_analytical_profile_summary_t180',
                                                                to_table_name = 'biui_sampled_{}_consumer_analytical_profile_summary_t180'.format(
                                                                    str(int((100/sample_rate_percentage)*100))),
                                                                sample_rate = int(100/sample_rate_percentage),
                                                                string_atr = 'member_id',
                                                                extr_loc_by_table_name = True)

user_interactions_db_handler.random_sample_based_on_string_attr(from_table_name = 'biui_consumer_analytical_profile_summary_t030',
                                                                to_table_name = 'biui_sampled_{}_consumer_analytical_profile_summary_t030'.format(
                                                                    str(int((100/sample_rate_percentage)*100))),
                                                                sample_rate = int(100/sample_rate_percentage),
                                                                string_atr = 'member_id',
                                                                extr_loc_by_table_name = True)

user_interactions_db_handler.random_sample_based_on_string_attr(from_table_name = 'biui_consumer_analytical_profile_summary_t007',
                                                                to_table_name = 'biui_sampled_{}_consumer_analytical_profile_summary_t007'.format(
                                                                    str(int((100/sample_rate_percentage)*100))),
                                                                sample_rate = int(100/sample_rate_percentage),
                                                                string_atr = 'member_id',
                                                                extr_loc_by_table_name = True)

user_interactions_db_handler.random_sample_based_on_string_attr(from_table_name = 'biui_consumer_analytical_profile_summary_t001',
                                                                to_table_name = 'biui_sampled_{}_consumer_analytical_profile_summary_t001'.format(
                                                                    str(int((100/sample_rate_percentage)*100))),
                                                                sample_rate = int(100/sample_rate_percentage),
                                                                string_atr = 'member_id',
                                                                extr_loc_by_table_name = True)

2019-08-07 17:53:43.453 INFO athena_util.py.441 - s3://datascience-workspace-dev/buyer-intent/user_interactions
2019-08-07 17:53:43.550 INFO athena_util.py.462 - 2019-08-07 17:53:43.550292
2019-08-07 17:53:45.111 INFO athena_util.py.464 - 2019-08-07 17:53:45.111585
2019-08-07 17:53:45.168 INFO athena_util.py.441 - s3://datascience-workspace-dev/buyer-intent/user_interactions
2019-08-07 17:53:45.227 INFO athena_util.py.462 - 2019-08-07 17:53:45.227940
2019-08-07 17:53:47.304 INFO athena_util.py.464 - 2019-08-07 17:53:47.304191
2019-08-07 17:53:47.454 INFO athena_util.py.441 - s3://datascience-workspace-dev/buyer-intent/user_interactions
2019-08-07 17:53:47.518 INFO athena_util.py.462 - 2019-08-07 17:53:47.518577
2019-08-07 17:53:49.066 INFO athena_util.py.464 - 2019-08-07 17:53:49.066829
2019-08-07 17:53:49.157 INFO athena_util.py.441 - s3://datascience-workspace-dev/buyer-intent/user_interactions
2019-08-07 17:53:49.219 INFO athena_util.py.462 - 2019-08-07 17:53:49.219537
2019-08-07 17

0

In [9]:
s3_data_path_180 = 's3://datascience-workspace-dev/buyer-intent/user_interactions/tables/{}'.format(
    'biui_sampled_{}_consumer_analytical_profile_summary_t180'.format(str(int((100/sample_rate_percentage)*100))))

s3_data_path_030 = 's3://datascience-workspace-dev/buyer-intent/user_interactions/tables/{}'.format(
    'biui_sampled_{}_consumer_analytical_profile_summary_t030'.format(str(int((100/sample_rate_percentage)*100))))

s3_data_path_007 = 's3://datascience-workspace-dev/buyer-intent/user_interactions/tables/{}'.format(
    'biui_sampled_{}_consumer_analytical_profile_summary_t007'.format(str(int((100/sample_rate_percentage)*100))))

s3_data_path_001 = 's3://datascience-workspace-dev/buyer-intent/user_interactions/tables/{}'.format(
    'biui_sampled_{}_consumer_analytical_profile_summary_t001'.format(str(int((100/sample_rate_percentage)*100))))

In [10]:
df_180 = pq.ParquetDataset(s3_data_path_180, filesystem=s3).read_pandas().to_pandas()
df_030 = pq.ParquetDataset(s3_data_path_030, filesystem=s3).read_pandas().to_pandas()

In [11]:
df_180 = df_180.drop(columns = ['snapshot_date_mst_yyyymmdd','most_recent_ldp_visit_date','ldp_dominant_zip'] )
df_030 = df_030.drop(columns = ['snapshot_date_mst_yyyymmdd','most_recent_ldp_visit_date','ldp_dominant_zip'] )

In [12]:
df_180_030 = pd.merge(left = df_030, right = df_180, left_on = 'member_id', right_on = 'member_id')
print(df_180_030['member_id'].unique())

In [15]:
#Get correlation matrix for possible dimension reduction
df_180_030 = df_180_030.drop(columns=['member_id'])
df_180_030 = df_180_030.corr()
df_180_030.style.background_gradient(cmap='coolwarm')

,total_ldp_page_views_x,median_ldp_page_views_x,average_ldp_page_views_x,average_ldp_dwell_time_seconds_x,total_ldp_dwell_time_seconds_x,median_ldp_dwell_time_seconds_x,minimum_ldp_dwell_time_seconds_x,maximum_ldp_dwell_time_seconds_x,total_distinct_listings_viewed_x,total_listings_viewed_x,median_distinct_listings_viewed_x,average_distinct_listings_viewed_x,median_listings_viewed_x,average_total_listings_viewed_x,total_ldp_page_views_y,median_ldp_page_views_y,average_ldp_page_views_y,average_ldp_dwell_time_seconds_y,total_ldp_dwell_time_seconds_y,median_ldp_dwell_time_seconds_y,minimum_ldp_dwell_time_seconds_y,maximum_ldp_dwell_time_seconds_y,total_distinct_listings_viewed_y,total_listings_viewed_y,median_distinct_listings_viewed_y,average_distinct_listings_viewed_y,median_listings_viewed_y,average_total_listings_viewed_y
total_ldp_page_views_x,1,0.109485,0.309823,0.0222739,0.805375,-0.0553537,-0.0574916,0.443205,0.87261,0.881617,0.87261,0.87261,0.881617,0.881617,0.554566,0.115203,0.279837,0.0289998,0.410998,-0.0350487,-0.0286485,0.228841,0.447634,0.465238,0.447634,0.447634,0.465238,0.465238
median_ldp_page_views_x,0.109485,1,0.817695,0.312303,0.0963411,0.371241,0.240028,0.116809,0.0163155,0.0175837,0.0163155,0.0163155,0.0175837,0.0175837,0.0124501,0.372555,0.374265,0.167082,0.013335,0.160303,0.100625,0.0514235,-0.0352512,-0.0344146,-0.0352512,-0.0352512,-0.0344146,-0.0344146
average_ldp_page_views_x,0.309823,0.817695,1,0.351865,0.326841,0.252093,0.181597,0.357542,0.161394,0.16449,0.161394,0.161394,0.16449,0.16449,0.13344,0.381871,0.564484,0.231096,0.136294,0.134474,0.0730023,0.18943,0.03626,0.0409332,0.03626,0.03626,0.0409332,0.0409332
average_ldp_dwell_time_seconds_x,0.0222739,0.312303,0.351865,1,0.176313,0.769367,0.66418,0.461224,-0.0262497,-0.025243,-0.0262497,-0.0262497,-0.025243,-0.025243,-0.00465002,0.128077,0.184881,0.432446,0.0729022,0.290342,0.188931,0.226714,-0.0361356,-0.034742,-0.0361356,-0.0361356,-0.034742,-0.034742
total_ldp_dwell_time_seconds_x,0.805375,0.0963411,0.326841,0.176313,1,-0.00591363,-0.0466718,0.674613,0.692244,0.703073,0.692244,0.692244,0.703073,0.703073,0.489805,0.112982,0.310958,0.161893,0.522259,0.0152572,-0.0261093,0.353168,0.389246,0.406488,0.389246,0.389246,0.406488,0.406488
median_ldp_dwell_time_seconds_x,-0.0553537,0.371241,0.252093,0.769367,-0.00591363,1,0.712555,0.10107,-0.0861681,-0.0858209,-0.0861681,-0.0861681,-0.0858209,-0.0858209,-0.0607842,0.113159,0.0825363,0.289235,-0.0238822,0.319488,0.210481,0.0370035,-0.0742472,-0.0749614,-0.0742472,-0.0742472,-0.0749614,-0.0749614
minimum_ldp_dwell_time_seconds_x,-0.0574916,0.240028,0.181597,0.66418,-0.0466718,0.712555,1,0.0269995,-0.0778031,-0.0775764,-0.0778031,-0.0778031,-0.0775764,-0.0775764,-0.0580697,0.0800033,0.0328205,0.204038,-0.045842,0.250338,0.277001,8.80602e-05,-0.0649389,-0.0660129,-0.0649389,-0.0649389,-0.0660129,-0.0660129
maximum_ldp_dwell_time_seconds_x,0.443205,0.116809,0.357542,0.461224,0.674613,0.10107,0.0269995,1,0.373352,0.376756,0.373352,0.373352,0.376756,0.376756,0.291528,0.105492,0.315784,0.297628,0.358288,0.0487858,-0.0032279,0.497846,0.219966,0.229796,0.219966,0.219966,0.229796,0.229796
total_distinct_listings_viewed_x,0.87261,0.0163155,0.161394,-0.0262497,0.692244,-0.0861681,-0.0778031,0.373352,1,0.996603,1,1,0.996603,0.996603,0.611082,0.0428545,0.168381,-0.0236848,0.452498,-0.0660724,-0.0390321,0.209782,0.599308,0.616343,0.599308,0.599308,0.616343,0.616343
total_listings_viewed_x,0.881617,0.0175837,0.16449,-0.025243,0.703073,-0.0858209,-0.0775764,0.376756,0.996603,1,0.996603,0.996603,1,1,0.610464,0.0436598,0.170433,-0.0224338,0.453718,-0.0655239,-0.0389836,0.210522,0.594607,0.613422,0.594607,0.594607,0.613422,0.613422


In [19]:
df_007 = pq.ParquetDataset(s3_data_path_007, filesystem=s3).read_pandas().to_pandas()
df_007 = df_007.drop(columns = ['snapshot_date_mst_yyyymmdd','most_recent_ldp_visit_date','ldp_dominant_zip'] )

In [20]:
df_030_007 = pd.merge(left = df_030, right = df_007, left_on = 'member_id', right_on = 'member_id')
print(df_030_007['member_id'].unique())

['7830050' '553c32571cde56e2130003c4' '5cd254aaca2bb67833060051' ...
 '59e249d1ed510cfdf504a29d' '5a27346febdbcf230220a4a8'
 '5d2e105062cb2e90930038f3']


In [21]:
#Get correlation matrix for possible dimension reduction
df_030_007 = df_030_007.drop(columns=['member_id'])
df_030_007 = df_030_007.corr()
df_030_007.style.background_gradient(cmap='coolwarm')

,total_ldp_page_views_x,median_ldp_page_views_x,average_ldp_page_views_x,average_ldp_dwell_time_seconds_x,total_ldp_dwell_time_seconds_x,median_ldp_dwell_time_seconds_x,minimum_ldp_dwell_time_seconds_x,maximum_ldp_dwell_time_seconds_x,total_distinct_listings_viewed_x,total_listings_viewed_x,median_distinct_listings_viewed_x,average_distinct_listings_viewed_x,median_listings_viewed_x,average_total_listings_viewed_x,total_ldp_page_views_y,median_ldp_page_views_y,average_ldp_page_views_y,average_ldp_dwell_time_seconds_y,total_ldp_dwell_time_seconds_y,median_ldp_dwell_time_seconds_y,minimum_ldp_dwell_time_seconds_y,maximum_ldp_dwell_time_seconds_y,total_distinct_listings_viewed_y,total_listings_viewed_y,median_distinct_listings_viewed_y,average_distinct_listings_viewed_y,median_listings_viewed_y,average_total_listings_viewed_y
total_ldp_page_views_x,1,0.130713,0.334268,0.0345669,0.80012,-0.0504655,-0.0491677,0.439656,0.867323,0.87682,0.867323,0.867323,0.87682,0.87682,0.67816,0.0819169,0.206313,0.0250571,0.564207,-0.0419995,-0.0493936,0.363277,0.624971,0.627117,0.624971,0.624971,0.627117,0.627117
median_ldp_page_views_x,0.130713,1,0.795101,0.359604,0.114549,0.433239,0.28357,0.126611,0.0213989,0.0229788,0.0213989,0.0213989,0.0229788,0.0229788,0.106327,0.446618,0.436628,0.171507,0.101985,0.150826,0.12559,0.110546,0.026366,0.0270614,0.026366,0.026366,0.0270614,0.0270614
average_ldp_page_views_x,0.334268,0.795101,1,0.418526,0.352737,0.28883,0.213917,0.382502,0.156536,0.160416,0.156536,0.156536,0.160416,0.160416,0.249479,0.4866,0.591504,0.228866,0.273197,0.140587,0.0980399,0.293123,0.12829,0.130037,0.12829,0.12829,0.130037,0.130037
average_ldp_dwell_time_seconds_x,0.0345669,0.359604,0.418526,1,0.210733,0.725692,0.586313,0.468111,-0.0290885,-0.0278436,-0.0290885,-0.0290885,-0.0278436,-0.0278436,0.025497,0.201554,0.226665,0.421787,0.14079,0.340592,0.286557,0.24027,-0.0209671,-0.0203111,-0.0209671,-0.0209671,-0.0203111,-0.0203111
total_ldp_dwell_time_seconds_x,0.80012,0.114549,0.352737,0.210733,1,-2.6242e-05,-0.0436587,0.673452,0.6773,0.68878,0.6773,0.6773,0.68878,0.68878,0.536351,0.0806517,0.221232,0.126436,0.684388,-8.70932e-05,-0.0358211,0.509764,0.475154,0.478882,0.475154,0.475154,0.478882,0.478882
median_ldp_dwell_time_seconds_x,-0.0504655,0.433239,0.28883,0.725692,-2.6242e-05,1,0.657939,0.0775373,-0.0832453,-0.082955,-0.0832453,-0.0832453,-0.082955,-0.082955,-0.0342422,0.160059,0.129503,0.314968,0.00886659,0.326209,0.296946,0.0584773,-0.0591285,-0.0590813,-0.0591285,-0.0591285,-0.0590813,-0.0590813
minimum_ldp_dwell_time_seconds_x,-0.0491677,0.28357,0.213917,0.586313,-0.0436587,0.657939,1,0.00685184,-0.0661712,-0.0660672,-0.0661712,-0.0661712,-0.0660672,-0.0660672,-0.0359823,0.0996012,0.0757756,0.235888,-0.026431,0.233609,0.312301,0.0096426,-0.0478972,-0.0480235,-0.0478972,-0.0478972,-0.0480235,-0.0480235
maximum_ldp_dwell_time_seconds_x,0.439656,0.126611,0.382502,0.468111,0.673452,0.0775373,0.00685184,1,0.354136,0.358041,0.354136,0.354136,0.358041,0.358041,0.301052,0.118509,0.25328,0.238947,0.453239,0.0804418,0.0252987,0.548606,0.247302,0.249096,0.247302,0.247302,0.249096,0.249096
total_distinct_listings_viewed_x,0.867323,0.0213989,0.156536,-0.0290885,0.6773,-0.0832453,-0.0661712,0.354136,1,0.996516,1,1,0.996516,0.996516,0.598944,0.00181195,0.089322,-0.0191546,0.472542,-0.0680767,-0.066542,0.291105,0.703548,0.701859,0.703548,0.703548,0.701859,0.701859
total_listings_viewed_x,0.87682,0.0229788,0.160416,-0.0278436,0.68878,-0.082955,-0.0660672,0.358041,0.996516,1,0.996516,0.996516,1,1,0.597873,0.00208827,0.0901736,-0.0185037,0.475405,-0.0679031,-0.0664318,0.292848,0.697614,0.697537,0.697614,0.697614,0.697537,0.697537


In [22]:
df_001 = pq.ParquetDataset(s3_data_path_001, filesystem=s3).read_pandas().to_pandas()
df_001 = df_001.drop(columns = ['snapshot_date_mst_yyyymmdd','most_recent_ldp_visit_date','ldp_dominant_zip'] )

In [23]:
df_007_001 = pd.merge(left = df_007, right = df_001, left_on = 'member_id', right_on = 'member_id')

In [24]:
#Get correlation matrix for possible dimension reduction
df_007_001 = df_007_001.drop(columns=['member_id'])
df_007_001 = df_007_001.corr()
df_007_001.style.background_gradient(cmap='coolwarm')

,total_ldp_page_views_x,median_ldp_page_views_x,average_ldp_page_views_x,average_ldp_dwell_time_seconds_x,total_ldp_dwell_time_seconds_x,median_ldp_dwell_time_seconds_x,minimum_ldp_dwell_time_seconds_x,maximum_ldp_dwell_time_seconds_x,total_distinct_listings_viewed_x,total_listings_viewed_x,median_distinct_listings_viewed_x,average_distinct_listings_viewed_x,median_listings_viewed_x,average_total_listings_viewed_x,total_ldp_page_views_y,median_ldp_page_views_y,average_ldp_page_views_y,average_ldp_dwell_time_seconds_y,total_ldp_dwell_time_seconds_y,median_ldp_dwell_time_seconds_y,minimum_ldp_dwell_time_seconds_y,maximum_ldp_dwell_time_seconds_y,total_distinct_listings_viewed_y,total_listings_viewed_y,median_distinct_listings_viewed_y,average_distinct_listings_viewed_y,median_listings_viewed_y,average_total_listings_viewed_y
total_ldp_page_views_x,1,0.120246,0.311754,0.0256591,0.759263,-0.0537063,-0.053207,0.444187,0.876552,0.880211,0.876552,0.876552,0.880211,0.880211,0.556347,0.0951386,0.178373,0.0183583,0.412335,-0.0305775,-0.0394306,0.256457,0.496985,0.497593,0.496985,0.496985,0.497593,0.497593
median_ldp_page_views_x,0.120246,1,0.750037,0.400044,0.118404,0.442673,0.280266,0.151005,0.00878001,0.00951065,0.00878001,0.00878001,0.00951065,0.00951065,0.099003,0.361936,0.349357,0.123234,0.089698,0.106178,0.0900652,0.0956033,0.0171375,0.017299,0.0171375,0.0171375,0.017299,0.017299
average_ldp_page_views_x,0.311754,0.750037,1,0.416604,0.311299,0.283717,0.212262,0.362407,0.119312,0.121303,0.119312,0.119312,0.121303,0.121303,0.193713,0.379153,0.440828,0.147025,0.194601,0.0978019,0.0706773,0.194652,0.0827248,0.083124,0.0827248,0.0827248,0.083124,0.083124
average_ldp_dwell_time_seconds_x,0.0256591,0.400044,0.416604,1,0.232983,0.801648,0.678785,0.465512,-0.0396922,-0.0389925,-0.0396922,-0.0396922,-0.0389925,-0.0389925,0.018609,0.164839,0.172951,0.294567,0.136067,0.244635,0.211654,0.201998,-0.0231303,-0.0230004,-0.0231303,-0.0231303,-0.0230004,-0.0230004
total_ldp_dwell_time_seconds_x,0.759263,0.118404,0.311299,0.232983,1,0.0155123,-0.0347146,0.732082,0.633716,0.64007,0.633716,0.633716,0.64007,0.64007,0.420025,0.0978342,0.185438,0.108812,0.522333,0.0186921,-0.0171376,0.381621,0.356357,0.357309,0.356357,0.356357,0.357309,0.357309
median_ldp_dwell_time_seconds_x,-0.0537063,0.442673,0.283717,0.801648,0.0155123,1,0.675492,0.125739,-0.0857982,-0.085978,-0.0857982,-0.0857982,-0.085978,-0.085978,-0.0294719,0.114025,0.0947007,0.208064,0.0224283,0.210192,0.197137,0.0650286,-0.0509,-0.0509503,-0.0509,-0.0509,-0.0509503,-0.0509503
minimum_ldp_dwell_time_seconds_x,-0.053207,0.280266,0.212262,0.678785,-0.0347146,0.675492,1,0.0404408,-0.0708389,-0.0711282,-0.0708389,-0.0708389,-0.0711282,-0.0711282,-0.0317952,0.0680757,0.0548722,0.156672,-0.0113499,0.149894,0.195889,0.0236254,-0.0427567,-0.0428285,-0.0427567,-0.0427567,-0.0428285,-0.0428285
maximum_ldp_dwell_time_seconds_x,0.444187,0.151005,0.362407,0.465512,0.732082,0.125739,0.0404408,1,0.348374,0.351706,0.348374,0.348374,0.351706,0.351706,0.254759,0.128816,0.206624,0.186817,0.383901,0.0839672,0.0361251,0.397039,0.198345,0.198979,0.198345,0.198345,0.198979,0.198979
total_distinct_listings_viewed_x,0.876552,0.00878001,0.119312,-0.0396922,0.633716,-0.0857982,-0.0708389,0.348374,1,0.99897,1,1,0.99897,0.99897,0.507747,0.0166938,0.0775019,-0.0145452,0.352032,-0.0523941,-0.0545207,0.204131,0.563008,0.563091,0.563008,0.563008,0.563091,0.563091
total_listings_viewed_x,0.880211,0.00951065,0.121303,-0.0389925,0.64007,-0.085978,-0.0711282,0.351706,0.99897,1,0.99897,0.99897,1,1,0.507961,0.0168423,0.0781043,-0.0143176,0.354009,-0.052563,-0.054769,0.20581,0.561895,0.56213,0.561895,0.561895,0.56213,0.56213


## **Summary of the Optimal Time Window**

It seems data presented in 90, 30, 7 and 1 data are orthagonals. So we may need all of them for modeling in future